In [1]:
import sys
sys.path.append('../..')
import numpy as np
from PIL import Image
import glob
# PyTorch
import torch
from torch import cuda
from torch.autograd import Variable
# DLBio and own scripts
from DLBio.pytorch_helpers import get_device
import transforms_data as td

In [2]:
class Config():
    DEVICE = get_device()
    DATASET_DIR = '../dataset/'
    MODEL_DIR = './models/ve_g_margin_3,0.pt'
    is_small_resize = False
    DATABASE_FOLDER = './embeddings/'

In [3]:
model = torch.load(Config.MODEL_DIR, map_location=torch.device(Config.DEVICE))
transformation = td.get_transform('siamese_valid_and_test', Config.is_small_resize)
# model.eval()

In [4]:
def pipeline(input_, preprocess):
    input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.is_small_resize)[0], td.get_resize(Config.is_small_resize)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)   
    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [5]:
for label in os.listdir(Config.DATASET_DIR):
    embeddings = []
    image_list = []
    for filename in glob.glob(Config.DATASET_DIR+label+'/*'): 
        img = Image.open(filename)
        img_processed = pipeline(img,transformation)
        image_list.append(img_processed)
    
    embeddings = np.array([model(Variable(i)).cpu() for i in image_list])
    
    np.save(Config.DATABASE_FOLDER+label+'.npy', embeddings)    